In [1]:
import pandas as pd
from sim_classes import TeamStatGen, TeamProfile, GameManager
from config import game_setup, local_host, local_name, local_port, local_user
import matplotlib.pyplot as plt
import psycopg2
import warnings
from creds import local_db_password
warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [2]:
season = '2023'
iterations = 50

In [3]:
fetch_games_sql = f"""
select 
    id,
    team_home,
    team_away,
    score_home,
    score_away,
    season_round
from ffl.fixtures 
where season = '{season}'
"""
real_results_df = pd.read_sql(fetch_games_sql, conn)
# real_results_df.head(1)

In [4]:
def aggregate_results(df, id_passthrough):
    home_win = 0
    away_win = 0
    tie_game = 0
    for index, row in df.iterrows():
        if row['score_home'] > row['score_away']:
            home_win += 1
        elif row['score_home'] < row['score_away']:
            away_win += 1
        else:
            tie_game += 1
    
    return id_passthrough, home_win, away_win, tie_game

sim block below

In [5]:
simulation_schedule = real_results_df['id']
sim_results_df = pd.DataFrame(columns=['id', 'home_win', 'away_win', 'tie_game'])

for i in simulation_schedule:
    team_stats = TeamStatGen(i)
    home_team = TeamProfile(team_stats.configure_team_stats('home'))
    away_team = TeamProfile(team_stats.configure_team_stats('away'))
    result_df = pd.DataFrame(columns=['score_home', 'score_away'])
    for j in range(iterations):
        game = GameManager(home_team, away_team, game_setup)
        result_df.loc[len(result_df)] = game.run_game('scores') #type: ignore

    sim_results_df.loc[len(sim_results_df)] = (aggregate_results(result_df, i)) #type: ignore


In [6]:
assess_sim_result = []
assess_real_result = []
assess_baseline_result = []
for index, row, in sim_results_df.iterrows():
    if row['home_win'] > row['away_win'] and row['home_win'] > row['tie_game']:
        assess_sim_result.append('win')
    elif row['away_win'] > row['home_win'] and row['away_win'] > row['tie_game']:
        assess_sim_result.append('loss')
    elif row['tie_game'] > row['home_win'] and row['tie_game'] > row['away_win']:
        assess_sim_result.append('tie')
    else:
        assess_sim_result.append('tie')

for index, row in real_results_df.iterrows():
    assess_baseline_result.append('win')

    if row['score_home'] > row['score_away']:
        assess_real_result.append('win')
    elif row['score_away'] > row['score_home']:
        assess_real_result.append('loss')
    elif row['score_home'] == row['score_away']:
        assess_real_result.append('tie')
def generate_conf_matrix(pred_results, real_results):
    pw_tw = 0
    pw_tl = 0
    pw_tt = 0
    pl_tw = 0
    pl_tl = 0
    pl_tt = 0
    pt_tw = 0
    pt_tl = 0
    pt_tt = 0

    if len(real_results) == len(pred_results):
        for game in range(len(pred_results)):
            if pred_results[game] == 'win' and real_results[game] == 'win':
                pw_tw +=1
            elif pred_results[game] == 'win' and real_results[game] == 'loss':
                pw_tl +=1
            elif pred_results[game] == 'win' and real_results[game] == 'tie':
                pw_tt +=1
            elif pred_results[game] == 'loss' and real_results[game] == 'win':
                pl_tw +=1
            elif pred_results[game] == 'loss' and real_results[game] == 'loss':
                pl_tl +=1
            elif pred_results[game] == 'loss' and real_results[game] == 'tie':
                pl_tt +=1
            elif pred_results[game] == 'tie' and real_results[game] == 'win':
                pt_tw +=1
            elif pred_results[game] == 'tie' and real_results[game] == 'loss':
                pt_tl +=1
            elif pred_results[game] == 'tie' and real_results[game] == 'tie':
                pt_tt +=1
        
        print(f'Total Accuracy:{(pw_tw + pl_tl + pt_tt) / len(real_results)}')
        # zero division error only intended to manage the baseline, "home always wins", data
        try:
            print(f'Win - Accuracy: {pw_tw / (pw_tw + pw_tl + pw_tt)} -- Recall: {(pw_tw / (pw_tw + pl_tw + pt_tw))}')
        except(ZeroDivisionError):
            print('Win Accuracy: 0 -- Recall: 0')
        try:
            print(f'Loss - Accuracy: {pl_tl/ (pl_tw + pl_tl + pl_tt)} -- Recall: {(pl_tl / (pw_tl + pl_tl + pt_tl))}')
        except(ZeroDivisionError):
            print('Loss Accuracy: 0 -- Recall: 0')
        try:
            print(f'Tie - Accuracy: {pt_tt/ (pt_tw + pt_tl + pt_tt)} -- Recall: {(pt_tt / (pw_tt + pl_tt + pt_tt))}')
        except(ZeroDivisionError):
            print('Tie Accuracy: 0 -- Recall: 0')

        return pd.DataFrame({
            'labels': ['true_win', 'true_loss', 'true_tie'],
            'pred_win':[pw_tw, pw_tl, pw_tt],
            'pred_loss':[pl_tw, pl_tl, pl_tt],
            'pred_tie':[pt_tw, pt_tl, pt_tt]
        })

In [7]:
sim_cm = generate_conf_matrix(assess_sim_result, assess_real_result)
sim_cm.head()

Total Accuracy:0.4921052631578947
Win - Accuracy: 0.5055762081784386 -- Recall: 0.7771428571428571
Loss - Accuracy: 0.4897959183673469 -- Recall: 0.3902439024390244
Tie - Accuracy: 0.23076923076923078 -- Recall: 0.036585365853658534


,labels,pred_win,pred_loss,pred_tie
0,true_win,136,33,6
1,true_loss,71,48,4
2,true_tie,62,17,3


In [8]:
base_cm = generate_conf_matrix(assess_baseline_result, assess_real_result)
base_cm.head()

Total Accuracy:0.4605263157894737
Win - Accuracy: 0.4605263157894737 -- Recall: 1.0
Loss Accuracy: 0 -- Recall: 0
Tie Accuracy: 0 -- Recall: 0


,labels,pred_win,pred_loss,pred_tie
0,true_win,175,0,0
1,true_loss,123,0,0
2,true_tie,82,0,0
